In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
from textblob import TextBlob
from nltk.corpus import stopwords
import gensim 
from nltk import sent_tokenize , word_tokenize
from gensim.utils import simple_preprocess
from tqdm import tqdm
import contractions
import string

In [2]:
df = pd.read_csv("combined_data.csv")
df

,label,text
0,1,ounce feather bowl hummingbird opec moment ala...
1,1,wulvob get your medircations online qnb ikud v...
2,0,computer connection from cnn com wednesday es...
3,1,university degree obtain a prosperous future m...
4,0,thanks for all your answers guys i know i shou...
...,...,...
83443,0,hi given a date how do i get the last date of ...
83444,1,now you can order software on cd or download i...
83445,1,dear valued member canadianpharmacy provides a...
83446,0,subscribe change profile contact us long term ...


In [3]:
df.isnull().sum()

label    0
text     0
dtype: int64

In [4]:
x_train = df['text']
y_train = df['label']

In [5]:
y_train

0        1
1        1
2        0
3        1
4        0
        ..
83443    0
83444    1
83445    1
83446    0
83447    1
Name: label, Length: 83448, dtype: int64

In [6]:
def preprocess(x):
    x = x.str.lower()
    def remove_tags(t):
        pattern = re.compile('<.*?>')
        return pattern.sub(r'',t)
    x = x.apply(remove_tags)
    def remove_urls(text):
        url_pattern = re.compile(r'https?://\S+|www\.\S+')
        text_without_urls = url_pattern.sub('', text)
        return text_without_urls
    x = x.apply(remove_urls)
    lst = stopwords.words('english')
    x.apply(lambda xi:[item for item in xi.split() if item not in lst]).apply(lambda xi:" ".join(xi))
    TEXT_CLEANING_RE = "[^a-zA-z\s]" # Matches character that are not letter or whitespace to remove them
    def clean_text(text):
        text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip() # Remove unwanted characters
        cleaned_text = re.sub('\s+', ' ', text).strip() # Normalize whitespace
        return cleaned_text
    x = x.apply(lambda xi: clean_text(xi))
    return x  

In [7]:
x_train = preprocess(x_train)

In [8]:
x_train

0        ounce feather bowl hummingbird opec moment ala...
1        wulvob get your medircations online qnb ikud v...
2        computer connection from cnn com wednesday esc...
3        university degree obtain a prosperous future m...
4        thanks for all your answers guys i know i shou...
                               ...                        
83443    hi given a date how do i get the last date of ...
83444    now you can order software on cd or download i...
83445    dear valued member canadianpharmacy provides a...
83446    subscribe change profile contact us long term ...
83447    get the most out of life viagra has helped mil...
Name: text, Length: 83448, dtype: object

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
x_train , x_test , y_train , y_test = train_test_split(x_train , y_train , test_size = 0.3 )

In [11]:
x_test ,x_val , y_test , y_val =  train_test_split(x_test , y_test , test_size = 0.5 )

In [12]:
x_train.size

58413

In [13]:
x_test.size

12517

In [14]:
x_val.size

12518

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(max_features = 15000)

In [16]:
x_train_tf = tf.fit_transform(x_train).toarray()
x_test_tf = tf.transform(x_test).toarray()

In [17]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(x_train_tf , y_train)
y_pred1 = gnb.predict(x_test_tf)
from sklearn.metrics import accuracy_score
accuracy_score(y_test , y_pred1)

0.973875529280179

In [18]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(x_train_tf, y_train)
y_pred2 = rf.predict(x_test_tf)
accuracy_score(y_test , y_pred2)

0.9863385795318367